# <b><font  color = "Navy" face="TimesNewRoman">PENGUMPULAN DATA</font></b>

## <font face="TimesNewRoman">1) Import modul </font>

In [3]:
from google_play_scraper import Sort, reviews

## <font face="TimesNewRoman">3) Scrap data dari Google Play</font>

### <font face="TimesNewRoman">(a) Informasi terkait data yang akan dikumpulkan </font>
1. Persiapan Awal Sebelum Memulai Scraping Data Google Playstore
   - Nama package = ``com.mobile.legends``
   - Bahasa = Indonesia (``id``)
   - Negara = Indonesia (``id``)
   - Kategori ulasan = ``NEWEST``
   - Jumlah data =  ``155300`` dari total ``155300``

### <font face="TimesNewRoman">(b) Memulai pengumpulan data</font>
Memulai dari scraping ``155300`` data

In [3]:
# List untuk menyimpan total data
total_reviews = []

# Mengambil data pertama
result, continuation_token = reviews(
    'com.mobile.legends',
    lang='id',
    country='id',
    sort=Sort.NEWEST,
    count=155300,
    filter_score_with=None
)

# Menambahkan hasil ke total_reviews
total_reviews.extend(result)

# Menampilkan jumlah baris data
print(f"Jumlah baris data = {len(result)}")

Jumlah baris data = 155300


### <font face="TimesNewRoman">(c) Melanjutkan pengumpulan data</font>

In [10]:
# Mengambil data berikutnya dengan continuation_token
result, _ = reviews(
    'com.mobile.legends',
    continuation_token=continuation_token # defaults to None(load from the beginning)
)

# Menambahkan data baru ke total_reviews
total_reviews.extend(result)
    
# Menampilkan total data yang sudah dikumpulkan
print(f"Total data yang sudah dikumpulkan: {len(total_reviews)}")

Total data yang sudah dikumpulkan: 924900


### <font face="TimesNewRoman">(d) Melihat isi <b>total_reviews</b>

In [12]:
# Menampilkan 5 review pertama dari total_reviews
print(total_reviews[:5])  # Menampilkan 5 item pertama

[{'reviewId': '48eed574-b912-46b7-ae4e-a5ded4fbe7fa', 'userName': 'Pengguna Google', 'userImage': 'https://play-lh.googleusercontent.com/EGemoI2NTXmTsBVtJqk8jxF9rh8ApRWfsIMQSt2uE4OcpQqbFu7f7NbTK05lx80nuSijCz7sc3a277R67g', 'content': 'game yg bikin gw salit kepala', 'score': 1, 'thumbsUpCount': 14, 'reviewCreatedVersion': '1.9.43.10342', 'at': datetime.datetime(2024, 12, 27, 9, 35, 39), 'replyContent': None, 'repliedAt': None, 'appVersion': '1.9.43.10342'}, {'reviewId': 'c23dc87e-ad8e-4504-b156-15d54a49fd52', 'userName': 'Pengguna Google', 'userImage': 'https://play-lh.googleusercontent.com/EGemoI2NTXmTsBVtJqk8jxF9rh8ApRWfsIMQSt2uE4OcpQqbFu7f7NbTK05lx80nuSijCz7sc3a277R67g', 'content': 'game busukkk kalhh teruss', 'score': 5, 'thumbsUpCount': 0, 'reviewCreatedVersion': None, 'at': datetime.datetime(2024, 12, 27, 9, 30, 33), 'replyContent': None, 'repliedAt': None, 'appVersion': None}, {'reviewId': '86a94d14-14e9-4b7f-99c7-bb558dba5804', 'userName': 'Pengguna Google', 'userImage': 'https:

## <font face="TimesNewRoman">4) Meyimpan data hasil scraping </font>

### <font face="TimesNewRoman">(a) Inisialisasi sesi spark dan membuat data frame baru

In [13]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
import datetime

# Inisialisasi Spark session
spark = SparkSession.builder.appName("GooglePlayReviews").getOrCreate()

# Mengonversi list of dictionaries menjadi DataFrame
df_reviews = spark.createDataFrame(total_reviews)

# Menyimpan DataFrame ke dalam file CSV
# df_reviews.write.csv('reviews.csv', header=True, mode='overwrite')

# Menampilkan DataFrame untuk memastikan
df_reviews.show()

24/12/28 10:17:31 WARN TaskSetManager: Stage 0 contains a task of very large size (121735 KiB). The maximum recommended task size is 1000 KiB.


+------------+-------------------+--------------------+-------------------+--------------------+--------------------+--------------------+-----+-------------+--------------------+---------------+
|  appVersion|                 at|             content|          repliedAt|        replyContent|reviewCreatedVersion|            reviewId|score|thumbsUpCount|           userImage|       userName|
+------------+-------------------+--------------------+-------------------+--------------------+--------------------+--------------------+-----+-------------+--------------------+---------------+
|1.9.43.10342|2024-12-27 09:35:39|game yg bikin gw ...|               null|                null|        1.9.43.10342|48eed574-b912-46b...|    1|           14|https://play-lh.g...|Pengguna Google|
|        null|2024-12-27 09:30:33|game busukkk kalh...|               null|                null|                null|c23dc87e-ad8e-450...|    5|            0|https://play-lh.g...|Pengguna Google|
|        null|2024-1

### <font face="TimesNewRoman"> Melihat beberapa atribut dari data frame

In [16]:
df_reviews_content = df_reviews.select('at','content')
df_reviews_content.show(5)

24/12/28 10:20:45 WARN TaskSetManager: Stage 3 contains a task of very large size (121735 KiB). The maximum recommended task size is 1000 KiB.


+-------------------+--------------------+
|                 at|             content|
+-------------------+--------------------+
|2024-12-27 09:35:39|game yg bikin gw ...|
|2024-12-27 09:30:33|game busukkk kalh...|
|2024-12-27 09:17:21|Game busuk bikin ...|
|2024-12-27 09:06:53|DAH GADA HARAPAN ...|
|2024-12-27 08:54:57|Kasih tim yg bene...|
+-------------------+--------------------+
only showing top 5 rows



### <font face="TimesNewRoman">(b) Menyimpan data frame

In [17]:
# Menggabungkan data menjadi satu file CSV
df_reviews.coalesce(1).write.csv('reviews.csv', header=True, mode='overwrite')

24/12/28 10:21:49 WARN TaskSetManager: Stage 4 contains a task of very large size (240625 KiB). The maximum recommended task size is 1000 KiB.


In [14]:
ls

Pengumpulan_Data.ipynb  Praproses.ipynb  kamus/  notebook/  reviews.csv/


In [15]:
!mv reviews.csv src

In [17]:
ls

Pengumpulan_Data.ipynb  Praproses.ipynb  kamus/  notebook/  src/


In [18]:
ls src

_SUCCESS  part-00000-b0389b7e-210a-4af8-80c3-72e0d7a74245-c000.csv


In [19]:
mv src/'part-00000-b0389b7e-210a-4af8-80c3-72e0d7a74245-c000.csv' 'reviews.csv'

In [21]:
ls

Pengumpulan_Data.ipynb  Praproses.ipynb  kamus/  notebook/  reviews.csv  src/


<hr>
<b><font face = "TimeNewRoman" size = 8; color = "Navy">AKHIR DARI PENGUMPULAN DATA</font></b>  <br>
<b><font color = "blue" face="TimesNewRoman" size="4"> Lanjut Praproses di kernel Praproses </font></b>